In [18]:
import pandas as pd
import string
import zeyrek

#Easier reading
pd.set_option('display.max_colwidth',None)
#Twitter dataset (one month)

#Getting input as the name of the file.
month_name = input("Please enter filename (e.g:jan20.csv): ")

twit_jan20_DF = pd.read_csv(month_name, sep=";", header=None)
#Transpose
twit_jan20_DFT = twit_jan20_DF.T
#twit_jan20_DFT.describe()

#ANEW Turkish dataset
anew_turkish = pd.read_excel("ANEW_Turkish.xlsx",sheet_name="Sheet1")
#column addition
anew_turkish.columns = ["isimler", "ing-karsiligi", "valence", "arousal", "dominance", "etiketler"]

#lemmatization object
analyzer = zeyrek.MorphAnalyzer()

#Good ol' mean, but does not cry when it is empty
def get_mean(alist):
    if len(alist) == 0:
        return 0
    return sum(alist)/len(alist)

#Ignores the zeros in the list
def get_mean_twit(alist):
    modded_length = 0
    for i in range(len(alist)):
        if alist[i] != 0:
            modded_length += 1
    
    if modded_length == 0:
        modded_length = 1
    return sum(alist)/modded_length

#This function takes a string (a Twit), lemmatize it, then assign statistical value of each word if it exists in ANEW dictionary.
def nlp_operations_VAD(a_twit):
    
    #Getting input twit
    string1 = str(a_twit)
    #Lowercase it
    string1 = string1.lower()
    #Split it (or call it tokenizing)
    split1 = [i for i in string1.split(' ')]

    t1 = []
    #This lemmatizes the words in the string
    for i in range(len(split1)):
        t1.append(analyzer.lemmatize(split1[i]))

    #creating lists for valence, arousal and dominance values' mean, so that row of twit can have these values    
    t1_V, t1_A, t1_D = [], [], []
    
    
    #iterate over list that holds the lemmatized words
    for i in range(len(t1)): 
        #iterate over lemmatization tuples
        for words in t1[i][0]:
            #Lemmas can be more than one so getting mean of interpretations could be useful.
            word_sum_V, word_sum_A, word_sum_D = [], [], []

            #Lemmtization products are in a list, take it by itself
            if type(words) == list:
                for i in words:
                    if (anew_turkish["isimler"] == i).any():
                        word_sum_V.append(anew_turkish.loc[anew_turkish["isimler"] == i]["valence"].iloc[0])
                        word_sum_A.append(anew_turkish.loc[anew_turkish["isimler"] == i]["arousal"].iloc[0])
                        word_sum_D.append(anew_turkish.loc[anew_turkish["isimler"] == i]["dominance"].iloc[0])

            #Just string, which is the word without lemmazitaion
            elif type(words) == str:

                if (anew_turkish["isimler"] == i).any():
                    word_sum_V.append(anew_turkish.loc[anew_turkish["isimler"] == i]["valence"].iloc[0])
                    word_sum_A.append(anew_turkish.loc[anew_turkish["isimler"] == i]["arousal"].iloc[0])
                    word_sum_D.append(anew_turkish.loc[anew_turkish["isimler"] == i]["dominance"].iloc[0])

            #Bunch up the words in mean to twit part.
            t1_V.append(get_mean(word_sum_V))
            t1_A.append(get_mean(word_sum_A))
            t1_D.append(get_mean(word_sum_D))


    return [round(get_mean_twit(t1_V), 2), round(get_mean_twit(t1_A), 2), round(get_mean_twit(t1_D), 2)]


#Some textual preprocessing
df1 = twit_jan20_DFT
#Add Twits column name
twit_jan20_DFT.columns = ["Twits"]
df1['Twits'] = df1['Twits'].astype("string")
df1['Twits'] = df1['Twits'].str.replace('@[A-Za-z0-9]+','')
df1['Twits'] = df1['Twits'].str.replace('http\S+','')
df1['Twits'] = df1['Twits'].str.replace('[^\w\s]+','')
df1['Twits'] = df1['Twits'].str.replace(' +', ' ')
df1["Twits"] = df1["Twits"].str.strip()
df1["Valence"] = None
df1["Arousal"] = None
df1["Dominance"] = None

#Going over the dataset and putting out the V, A, D columns
for idx in range(df1.shape[0]):
    #temp_VAD has 3 float numbers in it, returned from nlp_operations_VAD
    temp_VAD = nlp_operations_VAD(df1.iloc[idx, 0])
    #assigning Valence, Arousal, Dominance 
    df1.iloc[idx, 1] = temp_VAD[0]
    df1.iloc[idx, 2] = temp_VAD[1]
    df1.iloc[idx, 3] = temp_VAD[2]

#Output file name operations
x_ = month_name.split(".")
month_p_name = x_[0]+"_p."+x_[1]
df1.to_csv(month_p_name, header=True, sep=';', decimal='.')
print(month_p_name, "is created.")

Please enter filename (e.g:jan20.csv): oct20.csv


<ipython-input-18-8adeeb8a0e77>:98: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['Twits'] = df1['Twits'].str.replace('@[A-Za-z0-9]+','')
<ipython-input-18-8adeeb8a0e77>:99: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['Twits'] = df1['Twits'].str.replace('http\S+','')
<ipython-input-18-8adeeb8a0e77>:100: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['Twits'] = df1['Twits'].str.replace('[^\w\s]+','')
<ipython-input-18-8adeeb8a0e77>:101: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['Twits'] = df1['Twits'].str.replace(' +', ' ')


oct20_p.csv is created.


Please enter filename (e.g:jan20.csv): jan20.csv
jan20_p.csv


In [57]:
#df1["Twits"] = df1['Twits'].str.replace(r"\\r|\\n|\\t", "", regex=True)


index:  0
t1 len: 17
[[('rt', ['Rt'])], [('dünya', ['Dünya', 'dünya'])], [('sağlık', ['Sağlık', 'sağlık', 'sağ'])], [('örgütü', ['örgüt'])], [('corona', ['Corona'])], [('virüsü', ['virüs'])], [('nedeniyle', ['neden', 'nedeniyle'])], [('uluslararası', ['uluslararası'])], [('acil', ['acil'])], [('durum', ['duru', 'durum'])], [('ilan', ['ilân'])], [('ederken', ['etmek', 'eder'])], [('birleşik', ['birleşik'])], [('krallık', ['kral', 'Krallık'])], [('ülkede', ['ülke', 'Ülke'])], [('corona', ['Corona'])], [('virüsün', ['virüs'])]]
words:  rt
[[('rt', ['Rt'])], [('dünya', ['Dünya', 'dünya'])], [('sağlık', ['Sağlık', 'sağlık', 'sağ'])], [('örgütü', ['örgüt'])], [('corona', ['Corona'])], [('virüsü', ['virüs'])], [('nedeniyle', ['neden', 'nedeniyle'])], [('uluslararası', ['uluslararası'])], [('acil', ['acil'])], [('durum', ['duru', 'durum'])], [('ilan', ['ilân'])], [('ederken', ['etmek', 'eder'])], [('birleşik', ['birleşik'])], [('krallık', ['kral', 'Krallık'])], [('ülkede', ['ülke', 'Ülke'])], 

[3.06, 2.63, 3.05]

AttributeError: 'str' object has no attribute 'str'

[[('rt', ['Rt'])], [('_', ['_'])], [('dünya', ['dünya', 'Dünya'])], [('sağlık', ['sağ', 'sağlık', 'Sağlık'])], [('örgütü', ['örgüt'])], [('korona', ['koro'])], [('virüsü', ['virüs'])], [('için', ['için', 'içmek', 'iç'])], [('acil', ['acil'])], [('durum', ['duru', 'durum'])], [('ilan', ['ilân'])], [('etti', ['et', 'etmek'])], [('korona', ['koro'])], [('virüsü', ['virüs'])], [('için', ['için', 'içmek', 'iç'])], [('alınan', ['almak', 'alınmak'])], [('tedbirler', ['tedbir'])], [('araştırılsın', ['araştırmak'])], [('önergesi', ['önerge'])], [('akp', ['Akp'])]]


[[('rt', ['Rt'])],
 [('_', ['_'])],
 [('dünya', ['dünya', 'Dünya'])],
 [('sağlık', ['sağ', 'sağlık', 'Sağlık'])],
 [('örgütü', ['örgüt'])],
 [('korona', ['koro'])],
 [('virüsü', ['virüs'])],
 [('için', ['için', 'içmek', 'iç'])],
 [('acil', ['acil'])],
 [('durum', ['duru', 'durum'])],
 [('ilan', ['ilân'])],
 [('etti', ['et', 'etmek'])],
 [('korona', ['koro'])],
 [('virüsü', ['virüs'])],
 [('için', ['için', 'içmek', 'iç'])],
 [('alınan', ['almak', 'alınmak'])],
 [('tedbirler', ['tedbir'])],
 [('araştırılsın', ['araştırmak'])],
 [('önergesi', ['önerge'])],
 [('akp', ['Akp'])]]

<ipython-input-109-a06922ca3568>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test1df["Twits"] = test1df["Twits"].str.strip()


,Twits,Valence,Arousal,Dominance
0,7 bin kişilik gemide Korona virüsü şüphesi,None,None,None
1,7 bin kişilik gemide Korona virüsü şüphesi,None,None,None
2,RT _ Dünya Sağlık Örgütü korona virüsü için acil durum ilan etti Korona Virüsü için alınan tedbirler araştırılsın önergesi AKP,None,None,None
3,RT _HelpDesk ℹ Korona Virüsü kaynaklı durumu ve son gelişmeleri ulusal ve uluslararası otoritelerle değerlendirmek için Pekin Guangzh,None,None,None
4,Çinde korona virüs vakası nedeniyle ölü sayısı 170e yükseldi,None,None,None


,Twits,Valence,Arousal,Dominance
0,7 bin kişilik gemide Korona virüsü şüphesi,2.981667,2.550833,2.886667
1,7 bin kişilik gemide Korona virüsü şüphesi,2.981667,2.550833,2.886667
2,RT _ Dünya Sağlık Örgütü korona virüsü için acil durum ilan etti Korona Virüsü için alınan tedbirler araştırılsın önergesi AKP,3.037692,2.669615,3.092692
3,RT _HelpDesk ℹ Korona Virüsü kaynaklı durumu ve son gelişmeleri ulusal ve uluslararası otoritelerle değerlendirmek için Pekin Guangzh,3.081111,2.698333,3.202778
4,Çinde korona virüs vakası nedeniyle ölü sayısı 170e yükseldi,2.696667,2.570833,2.556667


In [112]:
"""
twit_jan20_DFT["Arousal"]=None
twit_jan20_DFT["Dominance"] = None

twit_jan20_DFT.head(5)
test1df.drop(test1df.columns[-1], axis=1, inplace=True)


#A piece of string
string1 = "abi selam naber"
#Lowercase it
string1 = string1.lower()
#Split it (or call it tokenizing)
split1 = [i for i in string1.split(' ')]

t1 = []
#This lemmatizes the words in the string
for i in range(len(split1)):
    t1.append(analyzer.lemmatize(split1[i]))
    
t1_V, t1_A, t1_D = [], [], []
#iterate over list that holds the lemmatized words
for i in range(len(t1)): #len(t1)
    #creating lists for valence, arousal and dominance values' mean, so that row of twit can have these values
    
    #iterate over lemmatization tuples
    for words in t1[i][0]:
        #Lemmas can be more than one so getting mean of interpretations could be useful.
        word_sum_V, word_sum_A, word_sum_D = [], [], []
        
        #Lemmtization products are in a list, take it by itself
        if type(words) == list:
            for i in words:
                if (anew_turkish["isimler"] == i).any():
                    word_sum_V.append(anew_turkish.loc[anew_turkish["isimler"] == i]["valence"].iloc[0])
                    word_sum_A.append(anew_turkish.loc[anew_turkish["isimler"] == i]["arousal"].iloc[0])
                    word_sum_D.append(anew_turkish.loc[anew_turkish["isimler"] == i]["dominance"].iloc[0])
   
        #Just string, which is the word without lemmazitaion
        elif type(words) == str:
            
            if (anew_turkish["isimler"] == i).any():
                word_sum_V.append(anew_turkish.loc[anew_turkish["isimler"] == i]["valence"].iloc[0])
                word_sum_A.append(anew_turkish.loc[anew_turkish["isimler"] == i]["arousal"].iloc[0])
                word_sum_D.append(anew_turkish.loc[anew_turkish["isimler"] == i]["dominance"].iloc[0])
        
        #Bunch up the words in mean to twit part.
        t1_V.append(get_mean(word_sum_V))
        t1_A.append(get_mean(word_sum_A))
        t1_D.append(get_mean(word_sum_D))

        
print(get_mean_twit(t1_V))
print(get_mean_twit(t1_A))
print(get_mean_twit(t1_D))

#print([get_mean_twit(t1_V), get_mean_twit(t1_A), get_mean_twit(t1_D)])
"""

#Removing stuff testing
"""
#Handles
abe = twit_jan20_DFT.iloc[3].astype("string")
print(abe)
abe = abe.str.replace('@[A-Za-z0-9]+', '')
print(abe)
#punctuation testing
abe = twit_jan20_DFT.iloc[3].astype("string")
print(abe)
abe = abe.str.replace('[^\w\s]+','')
print(abe)

#URLs
abe = twit_jan20_DFT.iloc[1].astype("string")
print(abe)
abe = abe.str.replace('http\S+','')
print(abe)

"""
"""
#Adding other columns
twit_jan20_DFT["Valence"] = None
twit_jan20_DFT["Arousal"]=None
twit_jan20_DFT["Dominance"] = None
"""


'\n#Adding other columns\ntwit_jan20_DFT["Valence"] = None\ntwit_jan20_DFT["Arousal"]=None\ntwit_jan20_DFT["Dominance"] = None\n'